In [1]:
%env BUCKET meliad_eu2
%env LOGURU_LEVEL INFO
%cd ~/dpr_jax
from src.data import load_from_seqio
import sys
sys.path.append("/home/ohadr/meliad2")
dataset = load_from_seqio("codeparrot","validation",repeat=False,limit=None)

env: BUCKET=meliad_eu2
env: LOGURU_LEVEL=INFO
/home/ohadr/dpr_jax


2024-01-21 10:51:58.228079: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2024-01-21 10:52:02.191814: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2024-01-21 10:52:02.191883: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


In [2]:
dataset.next()

{'nei_token_ids': array([    0,     0,     0, ...,  2262,   187, 50274], dtype=int32),
 'targets': array([    4, 45180, 12425, ...,    15,  8045,    64], dtype=int32),
 'book_id': -436174858,
 'nei_idx': array([ -1,  -1,  -1, ..., 196, 189,  50], dtype=int32),
 'nei_scores': array([     -inf,      -inf,      -inf, ..., 24.954105, 23.405523,
        23.344488], dtype=float32),
 'neig': array([ 16,  15,   0, ..., 208,  50,  19], dtype=int32)}

In [3]:
from datasets import load_dataset
from functools import partial
from transformers import AutoTokenizer
from torch.utils.data import DataLoader, IterableDataset
import numpy as np
import datasets


In [4]:


# tokenizer(text, truncation=True, padding="max_length", max_length=128)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
element = dataset.next()
targets = element["targets"]
chunks = targets.reshape([-1,64])

In [42]:
from functools import partial
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenize = partial(tokenizer,
                    return_attention_mask=True,
                    return_token_type_ids=False,
                    truncation=True,
                    max_length=128,
                    padding='max_length', return_tensors='np')
    
detokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
def encode_text(batch):
    input_ids = batch["input_ids"]
    detok_input_ids = detokenizer.batch_decode(input_ids)
    detok_passage = ["Passage: "+x for x in detok_input_ids]
    detok_question = ["Question: "+x for x in detok_input_ids]
    passage_dict = dict(tokenize(detok_passage))
    question_dict = dict(tokenize(detok_question))
    passage_dict = {"psgs_"+k:v for k,v in passage_dict.items()}
    question_dict = {"query_"+k:v for k,v in question_dict.items()}
    batch.update(**passage_dict,**question_dict)
    return batch


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [43]:
ds = datasets.Dataset.from_generator(lambda: map(lambda x:dict(input_ids=x),chunks))
ds = ds.map(encode_text, batched=True,remove_columns=ds.column_names)

Map:   0%|          | 0/209 [00:00<?, ? examples/s]

In [33]:
from torch.utils.data import DataLoader

In [47]:
import numpy as np
def package(result):
    keys = list(result[0].keys())
    batch = {}
    for key in keys:
        try:
            arr = np.array([res[key] for res in result])
            batch[key] = arr
        except ValueError:
            print([np.array(res[key]).shape for res in result])
            raise
    query = {"input_ids":batch['query_input_ids'],"attention_mask":batch['query_attention_mask']}
    psgs = {"input_ids":batch['psgs_input_ids'],"attention_mask":batch['psgs_attention_mask']}
    return query,psgs

In [48]:
dloader= DataLoader(ds,
                        batch_size=10,
                        collate_fn=lambda v: package(v)
                        )

In [52]:
batch = next(iter(dloader))

In [57]:
q_batch,p_batch = batch

In [59]:
model(**p_batch)[0][:, 0, :]

Array([[ 0.01021561,  0.19040376,  1.6352711 , ...,  1.1421033 ,
         0.95936346, -0.12786758],
       [-0.34261122,  0.53752893,  1.1948521 , ...,  0.94131815,
         0.85876745, -0.3351325 ],
       [-0.25598946,  0.38868222,  1.1916975 , ...,  0.75775605,
         0.65125734, -0.24994837],
       ...,
       [-0.18480799,  0.04126644,  1.4588346 , ...,  0.97078156,
         0.7898591 , -0.07024911],
       [ 0.02817565,  0.0380663 ,  1.590899  , ...,  0.91640073,
         0.8452434 , -0.3121782 ],
       [-0.3768375 ,  0.41357592,  1.4549042 , ...,  0.8485042 ,
         0.9712858 , -0.5253505 ]], dtype=float32)

In [29]:
from transformers import AutoConfig, AutoTokenizer, FlaxAutoModel

In [30]:
model = FlaxAutoModel.from_pretrained("/home/ohadr/dpr_jax/v7_n7_dscodeparrot_b20.95_wd0.01_steps100000/passage_encoder")

In [ ]:
params = ParamTuple(q_params=model.params,p_params=model.params)
state = RetrieverTrainState.create(apply_fn=model.__call__, params=params, tx=adamw)
q_reps = state.apply_fn(**queries, params=params.q_params, dropout_rng=q_dropout_rng, train=True)[0][:, 0, :]

In [28]:
ds

Dataset({
    features: ['passage_input_ids', 'passage_token_type_ids', 'passage_attention_mask', 'question_input_ids', 'question_token_type_ids', 'question_attention_mask'],
    num_rows: 209
})

In [2]:

p="https://huggingface.co/datasets/iohadrubin/nq/resolve/main/data/train-00000-of-00012-aebee16ac9d5ed6f.parquet"
train_dataset = load_dataset("parquet",data_files={"train":[p]},split="train")





train_data = train_dataset.map(
    partial(tokenize_examples,query_field="question",pos_field="positive_ctxs",neg_field="hard_negative_ctxs"),
    batched=False,
    num_proc=20,
    remove_columns=train_dataset.column_names,
    desc="Running tokenizer on train dataset",
)


In [8]:
train_dataset.filter(lambda x: len(x['positive_ctxs']) > 0 and len(x['hard_negative_ctxs']) >= 5,
                                         batched=False,
                                         num_proc=20)

Filter (num_proc=20):   0%|          | 0/4907 [00:00<?, ? examples/s]

Dataset({
    features: ['dataset', 'question', 'answers', 'positive_ctxs', 'negative_ctxs', 'hard_negative_ctxs'],
    num_rows: 4907
})

In [7]:
train_dataset

Dataset({
    features: ['dataset', 'question', 'answers', 'positive_ctxs', 'negative_ctxs', 'hard_negative_ctxs'],
    num_rows: 4907
})

In [3]:

class IterableDatasetWrapper(IterableDataset):
    def __init__(self, dataset):
        super(IterableDatasetWrapper).__init__()
        self.dataset = dataset
    def __iter__(self):
        while True:
            for x in self.dataset:
                yield x
            self.dataset = self.dataset.shuffle()

def package(result):
    keys = list(result[0].keys())
    batch = {}
    for key in keys:
        batch[key] = np.array([res[key] for res in result]).squeeze(-2)
    return batch   
def get_dataloader(data, batch_size):
    iterable = IterableDatasetWrapper(data) 
    dloader= DataLoader(iterable,
                            batch_size=batch_size,
                            collate_fn=lambda v: package(v),
                            num_workers=16, prefetch_factor=256,
                            )
    return dloader



In [5]:
train_data

Dataset({
    features: ['query_input_ids', 'psgs_input_ids'],
    num_rows: 4907
})

In [4]:
dloader = get_dataloader(train_data, 2)

In [68]:
b = next(iter(dloader))

In [69]:
b

{'query_input_ids': array([[ 101, 2502, 2210, 3658, 2161, 1016, 2129, 2116, 4178,  102,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
        [ 101, 2040, 6369, 3403, 2005, 1037, 2611, 2066, 2017,  102,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0]]),
 'psgs_input_ids': array([[[  101,  2502,  2210, ...,   102,     0,     0],
         [  101,  2210,  2111, ..., 18868,  1010,   102],
         [  101, 15883,  2007, ...,  2285,  2418,   102],
         ...,
         [  101,  5487, 20996, ...,  1000,  1012,   102],
         [  101,  2502,  2567, ...,  1015,  1012,   102],
         [  101,  2129,  1045, ...,  2544,  1997,   102]],
 
        [[  101,  3403,  2005, ...,  2316,  1005,   102],
         [  101,  3403,  2005, ...,  2051,  1000,   102],
         [  101,  3403,  2005, ...,  228

In [58]:
b.keys()

dict_keys(['query_input_ids', 'psgs_input_ids'])

In [61]:
b["query_input_ids"]

array([[ 101, 2502, 2210, 3658, 2161, 1016, 2129, 2116, 4178,  102,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [ 101, 2040, 6369, 3403, 2005, 1037, 2611, 2066, 2017,  102,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0]])

In [60]:
b["psgs_input_ids"].shape

(2, 10, 128)